## 4.6 Database Link in Python

### psycopg2

설치
```cmd
pip install psycopg2
pip install psycopg2-binary #위의 설치 에러 시 이 코드 실행
```
<br>

사용법
```python
conn = psycopg2.connect(
  host='address',
  dbname='database name',
  user='user name',
  password='password',
  port=port

cur = conn.cursor()
```

CRUD
```python
cur.execute(f'INSERT INTO test (id, press_name) VALUES ({id}, {press_name});')
conn.commit() # CREATE

cur.execute('SELECT * FROM test;')
result_one = cur.fetchone() # READ
result_many = cur.fetchmany() 
result_all = cur.fetchall() 

cur.execute(f'UPDATE test SET press_name={press_name} WHERE id > 5')
conn.commit() # UPDATE

cur.execute('DELETE FROM test WHERE press_name LIKE %조선%;')
conn.commit() # DELETE
```
<br>

pandas 내의 method를 통해서도 사용 가능
```python
pd.read_sql('SELECT * FROM test', conn)
```
<br>

Closer <br>
사용 후 연결 해제 <br>
```python
cur.close()
conn.close()
```
<br>

아래와 같이 사용 가능
```python
with conn.cursor() as cur:
  cur.execute(query)

conn.close()

with psycopg2.connect():
  with conn.cursor() as cur:
    cur.execute(query)
```

### sqlalchemy

설치
```cmd
pip install sqlalchemy
```
<br>

사용법
```python
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, DateTime

db_url = f'postgres+psycopg2://{USERNAME}:{PASSWORD}@{DB_HOST}:{PORT}/{DB_NAME}'
engine = create_engine(db_url, echo=True)

test.to_sql(
  'schema',
  engine,
  if_exists='append', # replace: 덮어쓰기
  index=False,
  chunksize=5000,
  dtypes={
    'id': Integer,
    'press_name': Text
  }
)
```

# 5.Crawling

스크레이핑(Scraping)이라고도 하며 웹 페이지 내의 데이터를 추출하는 것을 의미 <br>
데이터를 수집하기 위한 방법으로 많이 사용 <br>
크게 두 가지의 방법이 존재
1. 정적크롤링 <br>
정적 데이터를 수집하는 방법 <br>
정적 데이터란 페이지 내에 원하는 정보가 모두 드러남

2. 동적크롤링 <br>
동적 데이터를 수집하는 방법 <br>
동적 데이터란 클릭, 로그인 등의 행위를 통해 원하는 데이터에 접근 가능 <br>


|    |정적 크롤링|동적 크롤링|
|----|--------|--------|
|방법 |주소 사용  |브라우저 사용|
|수집 범위|제한적  |제한 없음|
|속도|매우 빠름|매우 느림
<br>

크롤링 시 사이트에서 크롤링을 허용하는지를 반드시 확인해야 함 <br>
robots.txt를 뒤에 붙여 확인 가능 <br>
강제는 아니나 이를 무시하면 추후 법률적 문제가 생길 수 있음
```
www.daum.net/robots.txt
User-agent: *
Disallow: /
```
*: All <br>
/: All Directories


## 5.1 HTTP

WWW(World Wide Web, W3) 상에서 정보를 주고받을 수 있는 프로토콜 <br>
클라이언트와 서버 사이에 이루어지는 요청/응답 프로토콜

### API
API(Application Programming Interface) <br>
- Application: 고유한 기능을 가진 모든 소프트웨어
- Interface: 두 애플리케이션 간의 규약 <br>
이 계약은 요청과 응답을 사용하여 두 애플리케이션이 서로 통신하는 방법을 정의합니다.

### REST
REST(Representational State Transfer): 자원을 이름으로 구분하여 해당 자원의 상태를 주고받는 것 <br>

REST 구성
1. 자원(Resource): HTTP URI
2. 자원에 대한 행위(Verb): HTTP Method
3. 자원에 대한 행위의 내용(Representations): HTTP Message Pay Load


- HTTP URI(Uniform Resource Identifier)를 통해 자원(Resource)을 명시
- HTTP Method(POST, GET, PUT, DELETE)를 사용하여 URI에 대한 CRUD Operation을 적용 <br>
<br>

HTTP Methods
- GET: 자원 검색
- POST: 자원 작성
- PUT: 자원 업데이트
- DELETE: 데이터 삭제
- HEAD: 자원 검색 (GET과 유사하나 상태 줄과 헤더만 반환)
- OPTIONS: 자원이 지원하고 있는 메소드의 취득
- PATCH: 자원 일부 수정 (PUT과 유사하나 일부만 수정)
- CONNECT: 자원의 터널 접속을 변경
- TRACE: 리소스에 대한 경로를 따라 메시지 루프백 테스트를 수행
<br>

HTTP Status
1. 1xx(Informational): 요청 처리중
2. 2xx(Successful): 요청 정상 처리 <br>
200: 요청 성공
3. 3xx(Redirection): 요청을 완료하려면 추가 행동이 필요
4. 4xx(Client Error): 클라이언트 오류, 잘못된 문법등으로 요청을 수행할 수 없음 <br>
400: Bad Request, 클라이언트의 잘못된 요청으로 서버가 요청을 처리할 수 없음 <br>
401: Unauthorized, 해당 리소스에 대한 인증이 필요함 <br>
403: Forbidden, 서버가 요청을 이해했지만 승인을 거부함 <br>
404: Not Found, 리소스를 찾을 수 없음 <br>
5. 5xx(Server Error): 서버 오류

### REST API
REST의 원리를 따르는 API <br>
※ RESTful: REST의 원리를 따르는 시스템

## 5.2 HTML
HTML(HyperText Markup Language)은 웹 페이지 표시를 위해 개발된 지배적인 마크업 언어 <br>
HTML은 웹 페이지 콘텐츠 안의 꺾쇠 괄호에 둘러싸인 "태그"로 되어있는 HTML 요소 형태로 작성 <br>
HTML은 웹 브라우저와 같은 HTML 처리 장치의 행동에 영향을 주는 자바스크립트, 본문과 그 밖의 항목의 외관과 배치를 정의하는 CSS 같은 스크립트를 포함하거나 불러올 수 있음 <br>
<br>
HTML 선택자: HTML에서는 다수의 동일한 태그가 존재하는데 각 태그를 구별할 수 있도록 선택자를 이용
```html
<div> 
	<div> 
      <a> c </a> 
      <span> c++ </span> 
    </div> 
    
    <div> 
      <a> java </a> 
      <span> python </span> 
    </div> 
</div>
```

```html
<div id="contents"> 
	<div class="data1"> 
      <span class="language"> c++ </span> 
      <span class="language"> java </span> 
      <span class="language"> python </span> 
  </div> 
    
  <div class="data2"> 
      <a class="framework"> tensorflow </a> 
      <a class="framework"> pytorch </a> 
      <a class="framework"> spring </a> 
  </div> 
</div>
```

## 5.3 정적크롤링

#### 5.3.1 라이브러리

##### 5.3.1.1 requests

requests: 파이썬용 http 라이브러리 <br>
reference: https://requests.readthedocs.io/en/latest/

메소드별 사용법
```python
GET: requests.get()
POST: requests.post()
PUT: requests.put()
DELETE: requests.delete()
```

```python
import requests

requests.get("https://jsonplaceholder.typicode.com/users/1")
```

###### Response Body

요청이 정상적으로 처리가 되면, response body에 요청한 데이터가 담겨져 옴. <br>
response body 크게 3가지 방식으로 읽을 수 있음 <br>
1. content: binary 원문을 읽음
```python
response.content
```
2. text: utf-8로 인코딩 된 문자열로 읽음
```python
response.text
```
3. json: 응답이 json이면 dict로 읽음
```python
response.json()
```



###### Request

param: 주소에 포함된 변수를 담음<br>
ex) https://www.naver.com/post/12345 <br>
-> 12345 <br>
query: 주소 바깥? 이후의 주소를 담음<br>
ex) https://www.naver.com/post/post_id=12345&id=1 <br>
-> 12345, 1
body: XML, JSON 등의 데이터를 담음, 주소에서는 확인 불가<br>
<br>

requests에서는 아래와 같이 사용
- get
```python
response = requests.get("https://naver.com/post", params={"post_id": "12345", "id": "1"})
```

- post, put: HTML 데이터 전송
```python
response = requests.get("https://naver.com/post", data={"post_id": "12345", "id": "1"})
```
json 형태로도 요청 가능
```python
response = requests.get("https://naver.com/post", json={"post_id": "12345", "id": "1"})
```

###### headers

일부 웹 사이트는 bot agent를 차단 <br>
이 경우 header의 user-agent를 아래와 같이 넘기면 해결 <br>
```python
requests.get("https://naver.com/post", headers={'User-Agent': 'Mozilla 5.0'})
```

##### 5.3.1.2 BeautifulSoup

BeautifulSoup: html, xml 등으로부터 원하는 정보를 가지고 올 수 있도록 하는 라이브러리 <br>
reference: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

```python
import requests
from bs4 import BeautifulSoup

response = requests.get(url)
bs = BeautifulSoup(response.text, 'lxml')
```

###### Parser

|parser|특징|설치|속도|사용방법|
|------|---|---|---|------|
|html.parser||기본|보통|BeautifulSoup(html_doc, 'html.parser')|
|lxml|xml 지원|lxml 필요|빠름|BeautifulSoup(html_doc, 'lxml')|
|xml|xml 지원|lxml 필요|빠름|BeautifulSoup(html_doc, 'xml')|
|html5lib|브라우저와 동일|html5lib 필요|느림|BeautifulSoup(html_doc, 'html5lib')|

###### find
속성과 값을 이용하여 원하는 값을 찾음

find: 매칭되는 값 중 상위 1개 반환
find_all: 매칭되는 전체 반환

특정 태그 추출
```python
soup.find_all('p') # p 태그 추출
```
<br>

특정 클래스 추출
```python
soup.find_all(class_='a') # a 클래스 추출
```
<br>

특정 태그와 class 추출
```python
soup.find_all('p', attrs={'class': 'a'}) # p 태그와 a 클래스 모두를 갖는 값 추출
```
<br>

특정 id 추출
```python
soup.find_all(id='b') # b id를 갖는 값 추출
```
<br>

###### select
CSS Selector로 태그를 찾아 반환 <br>
CSS에서 HTML을 태깅하는 방법을 활용 <br>
<br>
select_one: 매칭되는 값 중 상위 1개 반환 <br>
select: 매칭되는 전체 반환 <br>
<br>

특정 태그 추출
```python
soup.select('p') # p 태그 추출
```
<br>

특정 클래스 추출
```python
soup.select('.a') # a 클래스 추출
```
<br>

특정 태그와 class 추출
```python
soup.select('p.a') # p 태그와 a 클래스 모두를 갖는 값 추출
```
<br>

특정 id 추출
```python
soup.select('#b') # b id를 갖는 값 추출
```
<br>

특정 태그와 id 추출
```python
soup.select('p#b') # p 태그와 b id 모두를 갖는 값 추출
```
<br>

특정 태그와 class, id 모두 추출
```python
soup.select('p.a#b') # p 태그와 a 클래스 b id 모두 갖는 값 추출
```
<br>

특정 태그 아래에 있는 태그 찾기
```python
soup.select('div p') # div 아래 p태그가 있는 값 추출
soup.select('div > p') # div 바로 아래 p태그가 있는 값 추출
soup.select("div > #link") # div 바로 아래 link id가 있는 값 추출
```
<br>

형제 태그 찾기
```python
soup.select("#link + .sister") # link 태그와 형제 태그 중 바로 직후 1개
soup.select("#link ~ .sister") # link 태그와 형제 태그 중 뒤에 태그 전부
```
<br>

여러 태그 중 i번째 태그 추출
```python
soup.select('a:nth-of-type(i)') # 추출된 a태그 중 i번째 값 반환
```
<br>
<br>

정규표현식 활용 <br>
```python
soup.select('[class~=a]') # class 속성 중 a를 포함하는 태그
soup.select('a[href]') # a 태그 중 href 속성이 존재하는 태그
soup.select('a[href="https://www.naver.com"]') # a 태그 중 href 속성이 https://www.naver.com과 매칭되는 태그
soup.select('a[href^="https://"]') # a 태그 중 href 속성이 https://로 시작하는 태그
soup.select('a[href$="ac.kr"]') # a 태그 중 href 속성이 ac.kr로 끝나는 태그
soup.select('a[href*="naver"]') # a 태그 중 href 속성 중 naver를 가지는 태그
```

<br>
<br>

출력
```python
soup.strings # 값 반환
soup.stripped_strings # 공백을 제거한 값 반환
```

### 5.3.2 실습

##### 예제

```html
<div id="contents"> 
    <div class="data1"> 
      <span class="language"> c++ </span> 
      <span class="language"> java </span> 
      <span class="language"> python </span> 
  </div> 

  <div class="data2"> 
      <a class="framework"> tensorflow </a> 
      <a class="framework"> pytorch </a> 
      <a class="framework"> spring </a> 
  </div> 
</div>
```

In [6]:
response = '''<div id="contents"> 
	<div class="data1"> 
      <span class="language"> c++ </span> 
      <span class="language"> java </span> 
      <span class="language"> python </span> 
  </div> 
    
  <div class="data2"> 
      <a class="framework"> tensorflow </a> 
      <a class="framework"> pytorch </a> 
      <a class="framework"> spring </a> 
  </div> 
</div>'''

In [2]:
from bs4 import BeautifulSoup

In [7]:
bs = BeautifulSoup(response, 'lxml')

In [15]:
bs.select('div')[0].select('a') # a태그
bs.select('div')[0].select('.framework') # 클래스 framework
bs.select('div')[0].select('a.framework') # a태그 중 클래스값 framework

[<a class="framework"> tensorflow </a>,
 <a class="framework"> pytorch </a>,
 <a class="framework"> spring </a>]

In [22]:
bs.select('#contents') # id 가져오기
bs.select('div#contents') # div 태그 contents라는 id 가져오기

[<div id="contents">
 <div class="data1">
 <span class="language"> c++ </span>
 <span class="language"> java </span>
 <span class="language"> python </span>
 </div>
 <div class="data2">
 <a class="framework"> tensorflow </a>
 <a class="framework"> pytorch </a>
 <a class="framework"> spring </a>
 </div>
 </div>]

In [26]:
# span태그에서 java 값만 추출하기
bs.select('span')[1] 
bs.select('span:nth-of-type(2)')

[<span class="language"> java </span>]

In [27]:
bs.select('div a') #div태그 아래에 있는 a태그 가져오기(중간에 다른 태그가 있어도 가능!)

[<a class="framework"> tensorflow </a>,
 <a class="framework"> pytorch </a>,
 <a class="framework"> spring </a>]

In [28]:
bs.select('div > a') #div태그 바로 아래에 있는 a태그 가져오기(직접 하위관계의 태그만 가져옴)
bs.select('div > div') #div태그 바로 아래에 있는 div태그 가져오기

[<a class="framework"> tensorflow </a>,
 <a class="framework"> pytorch </a>,
 <a class="framework"> spring </a>]

#### 네이버 뉴스

In [116]:
!pip install psycopg2

In [117]:
import pandas as pd
import requests
import psycopg2
from tqdm import tqdm

In [47]:
response = requests.get(
    'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=023&date=20220820&page=1000',
    headers ={'User-Agent': 'Mozilla 5.0'})
bs = BeautifulSoup(response.text, 'lxml')

In [52]:
_last_page_num = bs.select('div.paging > strong')
last_page_num = int(_last_page_num[0].text)

In [53]:
last_page_num

6

In [62]:
[tag.text.replace('\n','').replace('\t','').strip() for tag in bs.select('li > dl > dt > a') if tag.text != '\n\n']

['철도 역세권 투자법 부동산 절세 전략… 전문가들 비법 공개',
 '[NOW] ‘카톡 손주’ 덕분에 스마트폰·키오스크 혼자서도 잘 써요',
 '한국갤럽 尹지지율 조사, 긍정평가 3%p 올라 28%',
 '“7명 살리고 떠난 아드님 뜻, 저희가 이을게요”',
 '“엄마가 내 어린시절 노출해 큰 피해 봤어요” 일본서 셰어런팅 역풍',
 '두 배 비싸게 만든 김원웅식 위인전… 김구 290쪽, 모친은 430쪽',
 '월성·북송… 대통령기록관 하루 두번 압수수색',
 '갤Z폴드·Z플립, 이번에도 우크라에선 ‘Z’ 빠졌다',
 '[백영옥의 말과 글](266) 글 쓰는 일']

제목 추출

In [70]:
all_titles = []
for page in range(1, last_page_num+1):
    response = requests.get(
        f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=023&date=20220820&page={page}',
        headers ={'User-Agent': 'Mozilla 5.0'})
    bs = BeautifulSoup(response.text, 'lxml')
    titles = [tag.text.replace('\n','').replace('\t','').strip() for tag in bs.select('li > dl > dt > a') if tag.text != '\n\n']
    all_titles += titles
all_titles

['저소득층 의료비 최대 5000만원 지원... 적용대상도 6대중증→모든 질환',
 '두 배 비싸게 만든 김원웅식 위인전… 김구 290쪽, 모친은 430쪽',
 '월성·북송… 대통령기록관 하루 두번 압수수색',
 '[NOW] ‘카톡 손주’ 덕분에 스마트폰·키오스크 혼자서도 잘 써요',
 '“尹도 당했다” 박근혜의 고추가루, 반기문의 퇴주 음복... 온라인 이미지 조작[더블클릭]',
 '아내 둔기로 내리치자 자녀들이 신고… 40대 남성 경찰에 체포',
 '정신질환 가족력 있는 여성은 조심… ‘이 병’ 위험 2배 높았다',
 '[박기자 쇄담] 유종의 미 노리는 다닐 메드베데프',
 '권성동 “김원웅, 광복회장 직함 달고 자기 가족 우상화…역사 팔아먹은 매국행위”',
 '다뉴브강 말라붙자...2차대전 때 침몰한 독일 군함들 드러났다',
 '펜스 前 부통령 “퇴임 때 기밀정보 가져간 적 없다”',
 '다크웹서 대마 판매, 2심 ‘범죄집단’ 인정... 총책 징역 7년',
 '상품권 싸게 판 뒤 잠적… 중고거래 사기친 20대 징역 4년',
 '김재원 “피해자는 윤석열, 피해호소인은 이준석”',
 '할인행사 비용 가맹점주에 떠넘긴 LG생건... 과징금 취소소송 패소',
 '“현수 바다에 빠뜨려야 하나” 이은해, 후배와 나눈 문자 속엔…',
 '북한, GDP 대비 군사비 지출 23%로 세계 1위...한국은?',
 '‘논리의 힘’ 없는 독재 정권이  ‘힘의 논리’로 표현의 자유 탄압',
 '[Market Watch] 아직 공격적인 연준 따라… 한은 다시 한번 ‘빅스텝’?',
 '美, 우크라에 1조 규모 무기 추가 지원…정찰 드론 스캔이글 첫 투입',
 '“벌금만 10조원”… 휠체어 납치 5년 뒤 나타난 中 억만장자 최후',
 'Conductor Goes Viral with Computer Game Theme Tunes',
 'Rags-to-Riches Stories Outsell Self-Help Books',
 'MLB 최고령 푸홀스의 ‘라스트 댄스’… 통산 700홈런 가능

In [72]:
def get_bs_data_from_naver_news(oid,date,page):
    response = requests.get(
        f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid={oid}&date={date}&page={page}',
        headers ={'User-Agent': 'Mozilla 5.0'})
    
    bs = BeautifulSoup(response.text, 'lxml')

    return bs
    

In [83]:
def get_titles_from_tags(bs: BeautifulSoup):
    return [tag.text.replace('\n','').replace('\t','').strip() for tag in bs.select('li > dl > dt > a') if tag.text != '\n\n']

In [74]:
bs = get_bs_data_from_naver_news('023','20220820',1000)
_last_page_num = bs.select('div.paging > strong')
last_page_num = int(_last_page_num[0].text)


In [89]:
all_titles = []
for page in range(1, last_page_num+1):
    response = get_bs_data_from_naver_news('023', '20220820', page)
    titles = get_titles_from_tags(response)
    all_titles += titles
all_titles

['저소득층 의료비 최대 5000만원 지원... 적용대상도 6대중증→모든 질환',
 '두 배 비싸게 만든 김원웅식 위인전… 김구 290쪽, 모친은 430쪽',
 '월성·북송… 대통령기록관 하루 두번 압수수색',
 '[NOW] ‘카톡 손주’ 덕분에 스마트폰·키오스크 혼자서도 잘 써요',
 '“尹도 당했다” 박근혜의 고추가루, 반기문의 퇴주 음복... 온라인 이미지 조작[더블클릭]',
 '아내 둔기로 내리치자 자녀들이 신고… 40대 남성 경찰에 체포',
 '정신질환 가족력 있는 여성은 조심… ‘이 병’ 위험 2배 높았다',
 '[박기자 쇄담] 유종의 미 노리는 다닐 메드베데프',
 '권성동 “김원웅, 광복회장 직함 달고 자기 가족 우상화…역사 팔아먹은 매국행위”',
 '다뉴브강 말라붙자...2차대전 때 침몰한 독일 군함들 드러났다',
 '펜스 前 부통령 “퇴임 때 기밀정보 가져간 적 없다”',
 '다크웹서 대마 판매, 2심 ‘범죄집단’ 인정... 총책 징역 7년',
 '상품권 싸게 판 뒤 잠적… 중고거래 사기친 20대 징역 4년',
 '김재원 “피해자는 윤석열, 피해호소인은 이준석”',
 '할인행사 비용 가맹점주에 떠넘긴 LG생건... 과징금 취소소송 패소',
 '“현수 바다에 빠뜨려야 하나” 이은해, 후배와 나눈 문자 속엔…',
 '북한, GDP 대비 군사비 지출 23%로 세계 1위...한국은?',
 '‘논리의 힘’ 없는 독재 정권이  ‘힘의 논리’로 표현의 자유 탄압',
 '[Market Watch] 아직 공격적인 연준 따라… 한은 다시 한번 ‘빅스텝’?',
 '美, 우크라에 1조 규모 무기 추가 지원…정찰 드론 스캔이글 첫 투입',
 '“벌금만 10조원”… 휠체어 납치 5년 뒤 나타난 中 억만장자 최후',
 'Conductor Goes Viral with Computer Game Theme Tunes',
 'Rags-to-Riches Stories Outsell Self-Help Books',
 'MLB 최고령 푸홀스의 ‘라스트 댄스’… 통산 700홈런 가능

In [85]:
def get_body_urls_from_tags(bs: BeautifulSoup):
    return[tag.attrs.get('href') for tag in bs.select('li > dl > dt > a') if tag.text != '\n\n']

In [103]:
all_titles = []
all_urls = []
for page in tqdm(range(1, last_page_num+1)):
    response = get_bs_data_from_naver_news('023', '20220820', page)
    titles = get_titles_from_tags(response)
    urls = get_body_urls_from_tags(response)
    all_titles += titles
    all_urls += urls


100%|██████████| 6/6 [00:00<00:00,  7.28it/s]


URL 추출

In [84]:
bs.select('li > dl > dt > a')[0].attrs.get('href') #딕셔너리 형태로 변환 키값으로 본문링크 추출

'https://n.news.naver.com/mnews/article/023/0003711006'

In [86]:
get_body_urls_from_tags(bs)

['https://n.news.naver.com/mnews/article/023/0003711006',
 'https://n.news.naver.com/mnews/article/023/0003711005',
 'https://n.news.naver.com/mnews/article/023/0003711004',
 'https://n.news.naver.com/mnews/article/023/0003711003',
 'https://n.news.naver.com/mnews/article/023/0003711002',
 'https://n.news.naver.com/mnews/article/023/0003711001',
 'https://n.news.naver.com/mnews/article/023/0003711000',
 'https://n.news.naver.com/mnews/article/023/0003710999',
 'https://n.news.naver.com/mnews/article/023/0003710998',
 'https://n.news.naver.com/mnews/article/023/0003710997',
 'https://n.news.naver.com/mnews/article/023/0003710996']

본문 추출

In [97]:
def get_contents_from_url(url:str):
    response = requests.get(url, headers = {'User-Agent' : 'Mozilla 5.0'})
    bs = BeautifulSoup(response.text, 'lxml')

    return bs.select('div#dic_area')[0].text

In [99]:
get_contents_from_url('https://n.news.naver.com/mnews/article/023/0003710996')

'\n스티븐 킹과 시드니 셸던의 팬이었던 어린 시절, 큰 의문이 있었다. 어떻게 한 인간이 이 길고 복잡한 소설을 그토록 줄기차게 써낼 수 있는지 궁금했던 것이다. 작가가 됐지만 여전히 나는 동료 작가들에게 작법이나 창작의 비밀 같은 것을 묻는다. 소설이 써지지 않을 때는 “첫 문장을 썼으니 반은 쓴 거나 매한가지!”라는 작가들의 자조 섞인 농담을 위안 삼기도 한다.하지만 정작 내가 소설 쓰는 일에 대해 잘 말하지 않게 된 건 글 쓰는 일이 사람들이 흔히 생각하는 ‘뮤즈’나 ‘영감’과 무관한 일이라는 걸 알아버렸기 때문이다. 아이디어나 작법이 중요한 건 사실이다. 하지만 작품을 꾸준히 쓰려면 태도가 더 중요하다. 글 쓰는 걸 좋아하는 것과 글을 써서 밥을 먹고 사는 일은 전혀 다른 종류의 일이기 때문이다.내 경우 좋아서 쓰는 것이라기보다, 쓰지 않으면 견딜 수 없으니 쓴다. 언제 써질지 모르니 불안해서 쓰고, 앞으로는 쓸 수 없을 거란 예감에 시달리니 쓰지 않을 수 없다. 간절함은 자신이 가진 능력을 증폭시키는 힘이 있다. 배우 윤여정도 “가장 연기가 잘될 때는 돈이 없을 때다”라고 말하지 않았던가. 책상 앞에 앉으면 막막함에 불안이 차오르지만, 일단 5매만 쓰자, 오늘은 썼으니 내일도 쓸 수 있을 거다, 라는 생각으로 꾸역꾸역 쓴다. 하루하루 그런 시간들이 모여 책이 된다.가슴 설레는 일을 하라는 스티브 잡스의 말은 우리를 꿈꾸게 하지만 현실에선 잘 적용되지 않는다. 글이 쓰고 싶어서 아침마다 눈이 번쩍 떠진다는 작가를 본 적이 없다. 그러니 “영감을 기다리는 건 아마추어고, 우리는 그냥 일을 하러 간다”는 소설가 필립 로스의 말을 성경처럼 새기고 ‘영감’이 아닌 ‘마감’의 힘으로 버티는 게 이 업계의 일이다.삶의 많은 부분이 실은 이런 힘에 의해 움직인다. 그러니 할 수 있고, 갈 수 있고, 쓸 수 있을 때 힘 내보자는 생각이 든다. 생전 박완서 선생님이 나이가 드니 책 못 읽는 게 가장 힘들다고 하셨던 말이 생각난다. 나도 노안이 오면 곧 못 읽을 때가 

In [95]:
response = requests.get('https://n.news.naver.com/mnews/article/023/0003711001', headers = {'User-Agent' : 'Mozilla 5.0'})
bs = BeautifulSoup(response.text, 'lxml')

In [96]:
bs.select('div#dic_area')[0].text

'\n19세때 장기기증 이태경군 아버지… 장기기증 서약한 고교생들 만났다\n\n\n\n지난 6일 오후 서울 서대문구 사랑의장기기증운동본부에서 2010년 뇌사 판정을 받은 후 7명에게 장기를 나눠주고 세상을 떠난 이태경군의 아버지 이대호(가운데)씨가 최근 장기기증 희망 등록을 한 고등학생 4명으로부터 꽃다발을 받고 있다. 이씨는“남을 돕는 것을 무척 좋아하던 태경이도 하늘나라에서 기뻐하고 있을 것”이라고 말했다./사랑의장기기증운동본부\t\t\t\t\t\t\t\t\t\t“장기를 기증한 아들에게 ‘대견하다’는 말을 직접 해주지 못했어요. 대신 여러분에게 오늘 그 말을 전해주고 싶습니다.”지난 6일 오후 3시쯤 서울 서대문구 사랑의장기기증운동본부 회의실. 이 말을 하는 이대호(60)씨의 눈시울이 붉어졌다. 이씨의 아들 이태경군은 열아홉이던 지난 2010년 8월 뇌와 척수에 종양이 생겨 뇌사 판정을 받았다. 결국 이군은 세상을 떠났지만 장기를 기증해 7명의 생명을 살렸다. 그 후로 12년이 지났다. 아버지 이씨는 사랑의장기기증운동본부 제안으로 지난 6일 당시 아들 또래인 고교생 4명을 만나 당시를 떠올리는 시간을 가졌다. 모두 만일의 사태가 생기면 ‘장기 기증을 하겠다’고 서약한 학생들이다.지난 2010년 당시 서울 송파구 가락시장에서 홀로 작은 채소 가게를 하던 이씨에게 아들은 든든한 지원군이자 둘도 없는 친구였다. 하지만 2010년 7월 어느 날, 학교에서 돌아온 아들이 “요즘 한쪽 눈이 잘 보이지 않아 학교에서 칠판 보기 불편하다”고 했다. 얼마 후 한 대학 병원에서 들은 검사 결과는 충격적이었다. ‘신경 교종증’이란 들어보지도 못한 병이었다. 뇌와 척수 내부에 있는 신경교세포에 종양이 생겼고, 이 종양이 시신경을 누르고 있어 한쪽 눈이 잘 보이지 않는 증상이 나타난 것이라고 했다. 이씨는 곧바로 병원에 입원시켜 치료를 시작했지만 태경군은 다음 날 의식을 잃고 쓰러졌다. 일주일 뒤 태경군은 의사에게 뇌사 판정을 받았다.\n\n\n\n故 이태경군\t\t\t\t\t\

데이터 통합

In [108]:
news = pd.DataFrame({
    'date': '20220820',
    'oid' : '023',
    'title': all_titles,
    'url': all_urls})

In [113]:
for row in tqdm(news.itertuples()):
    try:
        news.loc[row.Index, 'contents'] = get_contents_from_url(row.url)
    except:
        pass

115it [00:11, 10.44it/s]


In [114]:
news

,date,oid,title,url,contents
0,20220820,023,저소득층 의료비 최대 5000만원 지원... 적용대상도 6대중증→모든 질환,https://n.news.naver.com/mnews/article/023/000...,\n尹대통령 “정치복지서 약자복지로”\n\n\n\n윤석열 대통령이 19일 용산 대통...
1,20220820,023,"두 배 비싸게 만든 김원웅식 위인전… 김구 290쪽, 모친은 430쪽",https://n.news.naver.com/mnews/article/023/000...,"\n보훈처, 前광복회장 두번째 고발\n\n\n\n\t\t\t\t\t\t\t\t\t\..."
2,20220820,023,월성·북송… 대통령기록관 하루 두번 압수수색,https://n.news.naver.com/mnews/article/023/000...,\n文정부 청와대 윗선으로 향하는 검찰 수사\t\t\t\t\t\t\t검찰이 19일 ...
3,20220820,023,[NOW] ‘카톡 손주’ 덕분에 스마트폰·키오스크 혼자서도 잘 써요,https://n.news.naver.com/mnews/article/023/000...,\n노인·청년 이어주는 디지털 교육 인기\t\t\t\t\t\t\t경기 남양주에 사는...
4,20220820,023,"“尹도 당했다” 박근혜의 고춧가루, 반기문의 퇴주 음복... 온라인 이미지 조작[더...",https://n.news.naver.com/mnews/article/023/000...,\n\t\t\t\t\t\t\t\t\t\t윤석열 대통령은 18일 온라인에서 반대 진영...
...,...,...,...,...,...
110,20220820,023,“엄마가 내 어린시절 노출해 큰 피해 봤어요” 일본서 셰어런팅 역풍,https://n.news.naver.com/mnews/article/023/000...,\n\n\n\n\n'셰어런팅' 논란으로 역풍을 맞고 있는 일본 사이바라 리에코(58...
111,20220820,023,"두 배 비싸게 만든 김원웅식 위인전… 김구 290쪽, 모친은 430쪽",https://n.news.naver.com/mnews/article/023/000...,"\n보훈처, 前광복회장 두번째 고발\n\n\n\n\t\t\t\t\t\t\t\t\t\..."
112,20220820,023,월성·북송… 대통령기록관 하루 두번 압수수색,https://n.news.naver.com/mnews/article/023/000...,\n文정부 청와대 윗선으로 향하는 검찰 수사\t\t\t\t\t\t\t검찰이 19일 ...
113,20220820,023,"갤Z폴드·Z플립, 이번에도 우크라에선 ‘Z’ 빠졌다",https://n.news.naver.com/mnews/article/023/000...,\n\n\n\n\n삼성전자 우크라이나 홈페이지에 게시된 갤럭시Z폴드4와 갤럭시Z플립...


DB 연결

In [120]:
conn = psycopg2.connect(
  host='localhost',
  dbname='postgres',
  user='postgres',
  password='postgrespw',
  port=49154)

cur = conn.cursor()

In [ ]:
news.to_sql(
    'news',
    conn,
    if_exists = 'replace',
    index = False
)

#### 네이버 증권

#### 다음 증권